# 构建语音识别系统 - 解码与评测

**Help1： 本节课的内容主要为如何将模型的输出转化到识别结果（通过CTC解码和词典实现）和计算CER指标。**

## 如何从模型输出到识别文本

**Help2： 模型的输出维度需要让大家了解一下，notebook中的 (b, len, vocab_size)这三位代表的是什么。（开始训练模型的同学可能已经知道了，但这里要强调一下），分别为批大小，批次中序列最大长度（tensor需要保证一批数据中的长度一致，长度不够的要经过padding，因此我们的dataloader里面会有一个变量存储每个音频的长度），字典大小**

模型输出的结果为 (b, len, vocab_size)

**Help3： 贪婪搜索是对每一帧的预测选取概率最大的索引作为该帧的识别结果。对应的是notebook里面的greedy_search函数，这个函数是将模型的输出结果进行处理，对每一帧选取概率最大的位置，转换为字典对应的字符id。**

**该函数的输入为模型输出结果和该batch中每个音频的帧的长度。返回为一个len为batchsize的列表，每个元素为识别结果（字典中对应字符的id）的列表**

### greedy search

每一步选取预测概率最大的词

In [1]:
import torch
from tokenizer.tokenizer import Tokenizer
tokenizer = Tokenizer()

def greedy_search(ctc_probs: torch.Tensor, encoder_out_lens: torch.Tensor):
    batch_size, maxlen = ctc_probs.size()[:2]
    _, topk_index = ctc_probs.topk(1, dim=2)  # [batch, seq_len, 1]
    topk_index = topk_index.squeeze(-1)  # [batch, seq_len]
    encoder_out_lens = encoder_out_lens.cpu().tolist()

    hyps = []
    for i in range(batch_size):
        # 获取有效长度内的序列
        seq = topk_index[i, :encoder_out_lens[i]].cpu().tolist()
        
        # 移除重复字符和blank
        prev = None
        hyp = []
        for char in seq:
            if char == tokenizer.blk_id():  # 过滤blank
                prev = None
                continue
            if char != prev:  # 去重复
                hyp.append(char)
                prev = char
        hyps.append(hyp)
    
    return hyps

**Help4： example2.pt是我本地训练好的模型的一个输出样例，通过这个样例来帮助大家了解这个流程**

**下面的cell里面，打印的是该batch中的每个音频的长度**

In [2]:
tensordict = torch.load("./example2.pt")

pre = tensordict["pre"].to("cpu")
lens = tensordict["lens"].to("cpu")

print(lens)

tensor([46, 51, 44, 44, 41, 49, 48, 48, 74, 93, 44, 49, 50, 51, 58, 50])


In [3]:
res = greedy_search(pre, lens)

**Help5： 下面展示了经过贪婪搜索以后，对该batch的第一个音频的处理结果。**

**可以看到这里有一些特殊字符，这里需要大家做到下面的工作： 1.大家在上一节课了解了CTC的解码过程，请大家根据自己的理解来对这个输出进行处理；2.把特殊字符处理，例如\<sos\>和\<eos\>等**

In [4]:
from tokenizer.tokenizer import Tokenizer
tokenizer = Tokenizer()

print(res[0])

print(tokenizer.decode(res[0], ignore_special=False))

[2, 40, 188, 227, 247, 243, 375, 360, 32, 87, 251, 291, 282, 32, 141, 243, 55, 317, 3]
['<sos>', 'chen', 'pin', 'mao', 'hen', 'si', 'chi', 'zong', 'tiao', 'lian', 'jie', 'wei', 'pen', 'tiao', 'luan', 'si', 'zhua', 'nie', '<eos>']


**TODO：** 请大家根据CTC的解码思路，将模型的输出进行解码，移除重复字符和blank(上面的版本没有移除重复字符和blank)。

## 评测识别结果

**Help6： 计算CER指标。这个指标的计算涉及最小编辑距离（同学们应该在之前的算法课学习动态规划的时候实现过）。大家求S，D，I的时候，可以调Python包实现，也可以自己实现。最后计算总的CER的时候，请使用：$CER_{final} = \frac{S_{total}+D_{total}+I_{total}}{N_{total}}$，也就是大家需要计算总的S，D，I，N来求CER指标。**



这里我们采用字错率(CER, character error rate)来评测ASR系统的性能，计算公式如下:

$$CER = \frac{S+D+I}{N}$$

pre 代表模型预测， gt 代表正确识别结果。与最小编辑距离一致，将pre转化成gt，其中，S代表将 pre 转化成 gt 需要替换的数量，D 代表将 pre转化成 gt 需要删除的数量，I 代表将 pre 转化成 gt 需要插入的数量，N 代表gt 的长度。


**TODO：** 根据最小编辑距离求出 S，D，I，N ，完成ASR的CER指标评测

In [5]:
def calculate_cer(pre_tokens: list, gt_tokens: list) -> tuple:
    m, n = len(pre_tokens), len(gt_tokens)
    
    # 初始化动态规划表
    dp = [[0] * (n+1) for _ in range(m+1)]
    for i in range(m+1):
        dp[i][0] = i  # 删除所有pre字符
    for j in range(n+1):
        dp[0][j] = j  # 插入所有gt字符
        
    # 填充DP表
    for i in range(1, m+1):
        for j in range(1, n+1):
            if pre_tokens[i-1] == gt_tokens[j-1]:
                cost = 0
            else:
                cost = 1
            dp[i][j] = min(
                dp[i-1][j] + 1,    # 删除操作
                dp[i][j-1] + 1,    # 插入操作
                dp[i-1][j-1] + cost # 替换或匹配
            )
    
    # 回溯统计S/D/I
    i, j = m, n
    S = D = I = 0
    
    while i > 0 and j > 0:
        if pre_tokens[i-1] == gt_tokens[j-1]:
            # 匹配，无需操作
            i -= 1
            j -= 1
        else:
            # 优先选择操作次数最小的路径
            if dp[i][j] == dp[i-1][j-1] + 1:
                # 替换操作
                S += 1
                i -= 1
                j -= 1
            elif dp[i][j] == dp[i-1][j] + 1:
                # 删除操作
                D += 1
                i -= 1
            else:
                # 插入操作
                I += 1
                j -= 1
    
    # 处理剩余字符
    D += i  # 剩余pre字符需删除
    I += j  # 剩余gt字符需插入
    
    N = len(gt_tokens)
    cer = (S + D + I) / N if N != 0 else 0.0
    return cer, S, D, I, N

In [6]:
from data.dataloader import get_dataloader
from model.model import CTCModel
import torch
from utils.utils import to_device
from tqdm import tqdm

dev_dataloader = get_dataloader("./dataset/split/dev/wav.scp", "./dataset/split/dev/pinyin", 32, tokenizer, shuffle=False)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CTCModel(80, 256, tokenizer.size(), tokenizer.blk_id()).to(device)

checkpoint = torch.load("./model.pt", map_location=device)
model.load_state_dict(checkpoint['model'])

model.eval()

CTCModel(
  (subsampling): Subsampling(
    (subsampling): Conv2dSubsampling8(
      (conv): Sequential(
        (0): Conv2d(1, 256, kernel_size=(3, 3), stride=(2, 2))
        (1): ReLU()
        (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
        (3): ReLU()
        (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2))
        (5): ReLU()
      )
      (linear): Linear(in_features=2304, out_features=256, bias=True)
    )
  )
  (positional_encoding): RelPositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): ConformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x ConformerBlock(
        (ff1_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff1_linear1): Linear(in_features=256, out_features=1024, bias=True)
        (ff1_activation): SiLU()
        (ff1_dropout1): Dropout(p=0.1, inplace=False)
        (ff1_linear2): Linear(in_features=1024, out_features=256, bias=True)
        (ff1_dropout2): Dropout(p=0.1, inplace=False

In [7]:
def evaluate_model(dataloader, model, tokenizer, device='cpu'):
    all_refs = []
    all_hyps = []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="评估中"):
            batch = to_device(batch, device)
            audios = batch['audios']
            audio_lens = batch['audio_lens']
            texts = batch['texts']
            text_lens = batch['text_lens']
            
            encoder_out, _, encoder_out_lens = model(audios, audio_lens, texts, text_lens)
            
            hyps = greedy_search(encoder_out, encoder_out_lens)
            
            for i in range(len(text_lens)):
                ref = texts[i, :text_lens[i]].cpu().tolist()
                all_refs.append(ref)
                all_hyps.append(hyps[i])
    
    total_S = total_D = total_I = total_N = 0
    for ref, hyp in zip(all_refs, all_hyps):
        cer, S, D, I, N = calculate_cer(hyp, ref)
        total_S += S
        total_D += D
        total_I += I
        total_N += N
    
    final_cer = (total_S + total_D + total_I) / total_N if total_N > 0 else 1.0
    
    print(f"评估结果:")
    print(f"替换(S): {total_S}, 删除(D): {total_D}, 插入(I): {total_I}, 参考长度(N): {total_N}")
    print(f"CER: {final_cer:.4f} ({total_S+total_D+total_I}/{total_N})")

    print("\n样本对比:")
    for i in range(min(5, len(all_refs))):
        print(f"参考: {tokenizer.decode(all_refs[i])}")
        print(f"预测: {tokenizer.decode(all_hyps[i])}")
        print()
    
    return final_cer

In [8]:
cer = evaluate_model(dev_dataloader, model, tokenizer, device)
print(f"最终CER: {cer:.4f}")

评估中: 100%|██████████| 32/32 [00:04<00:00,  6.51it/s]


评估结果:
替换(S): 825, 删除(D): 46, 插入(I): 85, 参考长度(N): 19252
CER: 0.0497 (956/19252)

样本对比:
参考: ['yi', 'ge', 'nan', 'ren', 'tui', 'ran', 'de', 'zuo', 'zai', 'pang', 'bian', 'mu', 'guang', 'dai', 'zhi']
预测: ['yi', 'gen', 'nai', 'ren', 'tui', 'ran', 'de', 'zuo', 'zai', 'pang', 'bian', 'mu', 'guan', 'dai', 'zhi']

参考: ['xi', 'huan', 'ba', 'li', 'ao', 'de', 'shu', 'cha', 'zai', 'niu', 'zai', 'ku', 'de', 'qian', 'mian']
预测: ['xi', 'huan', 'ba', 'li', 'ao', 'de', 'shu', 'cha', 'zai', 'niu', 'zai', 'ku', 'de', 'qian', 'mian']

参考: ['zha', 'yan', 'yi', 'kan', 'xiang', 'qi', 'de', 'shi', 'guang', 'zhou', 'de', 'qu', 'hao', 'ling', 'e', 'er', 'ling']
预测: ['zhe', 'ye', 'yi', 'kan', 'xiang', 'qi', 'de', 'shi', 'guang', 'zhou', 'de', 'xu', 'hao', 'liu', 'e', 'er', 'ling']

参考: ['ci', 'qian', 'qing', 'hua', 'zi', 'guang', 'jiu', 'ceng', 'zao', 'yu', 'guo', 'tong', 'yang', 'de', 'wei', 'ji']
预测: ['ci', 'qian', 'qing', 'hua', 'zi', 'guang', 'jiu', 'ceng', 'zao', 'yu', 'guo', 'tong', 'yang', 'de', 'wei', 'ji